<a href="https://colab.research.google.com/github/s-miramontes/News_Filter/blob/master/notebooks/summarize_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarize Training Clusters with BERT 

In [0]:
# %%capture
# # Install the latest Tensorflow version.
# !pip3 install --upgrade tensorflow-gpu
# # Install TF-Hub.
# !pip3 install tensorflow-hub

In [0]:
# import statements 

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

from joblib import Parallel, delayed

import heapq
import operator

from absl import logging

import tensorflow as tf
import tensorflow_hub as hub

## Create Summaries for Clusters from Training Data

In [0]:
# import cluster data

clusters = pd.read_csv("news_filter/data/clusters.csv") 

In [0]:
# instantiate summarizer
model = Summarizer()

# function to return summary of each article in cluster
def make_summaries(cluster):
  result = {}
  for i in range(len(cluster.content)):
    summary = model(cluster.content[i], min_length=50, ratio=0.20) 
    result[i] = ''.join(summary)
  return result

In [0]:
# summarize every aritcle in clusters
cluster_summaries = []
for i in range(1,6):
  summaries = make_summaries(clusters[clusters.cluster_labels == i].reset_index())
  cluster_summaries.append(summaries)

In [0]:
cluster_summaries

## Create Summary of Summaries for each Cluster

In [0]:
# summarize summaries of each cluster 
summary_of_summaries = []
for summaries in cluster_summaries:
  summary = ' '.join(list(summaries.values()))
  summary_of_summaries.append(model(summary))

In [0]:
summary_of_summaries

## Output Final Summary

In [0]:
input_topics = ["Russian interference with election", "Hillary Clinton WikiLeaks email Benghazi", "Women's march highlights", "Bernie Sander's Campaign", "ICE immigration policies"]

summaries_df = pd.DataFrame({'topic': input_topics, 'summary': summary_of_summaries})
summaries_df
# summaries_df.to_csv('news_filter/data/summaries.csv', index=False)